In [607]:
import numpy as np

# generate data

In [608]:
# In real-world scenarios, learning how the data was generated is impractical. Do not rely on this function while doing research.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

# write your model class

In [609]:
from random import randint


# Modifying the overall structure is acceptable but not recommended
class SVM1:
    def __init__(self, dim, tol=1e-4, C=10):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim = dim
        self.w, self.b = np.zeros(dim), 0.0
        self.C, self.tol = C, tol

    def fit(self, X: np.ndarray, y: np.ndarray, eps=1e-4, max_round=10):
        """
        Fit the coefficients via your method1
        """
        N = X.shape[0]
        alpha = np.zeros((N, 1))
        b = 0.0
        K = X @ (X.transpose())
        E = np.zeros((N, 1))

        def gi(i):
            return K[i] @ (alpha * y) + b

        def calcE():
            nonlocal E
            for i in range(N):
                E[i] = gi(i) - y[i]

        calcE()

        def innerLoop(a1: int):
            nonlocal b

            def randa2():
                a2 = randint(0, N - 1)
                while a2 == a1:
                    a2 = randint(0, N - 1)
                return a2

            def run(a2: int):
                nonlocal b
                # calc new alpha1 and alpha2
                L, H = 0, 0
                if y[a1] != y[a2]:
                    L = max(0, alpha[a2] - alpha[a1])
                    H = min(self.C, self.C + alpha[a2] - alpha[a1])
                else:
                    L = max(0, alpha[a2] + alpha[a1] - self.C)
                    H = min(self.C, alpha[a2] + alpha[a1])
                if L >= H:
                    return 0

                def trunc(x):
                    return H if x > H else (L if x < L else x)

                K11, K22, K12 = K[a1][a1], K[a2][a2], K[a1][a2]
                eta = K11 + K22 - 2 * K12

                alpha2 = trunc(alpha[a2] + y[a2] * (E[a1] - E[a2]) / eta)
                alpha1 = alpha[a1] + y[a1] * y[a2] * (alpha[a2] - alpha2)

                # calc b
                b1new = (
                    -E[a1]
                    - y[a1] * K11 * (alpha1 - alpha[a1])
                    - y[a2] * K12 * (alpha2 - alpha[a2])
                    + b
                )
                b2new = (
                    -E[a2]
                    - y[a1] * K12 * (alpha1 - alpha[a1])
                    - y[a2] * K22 * (alpha2 - alpha[a2])
                    + b
                )
                if abs(alpha[a2] - alpha2) < eps:
                    return 0
                alpha[a1], alpha[a2] = alpha1, alpha2
                if 0 + eps < alpha[a1] < self.C - eps:
                    b = b1new
                elif 0 + eps < alpha[a2] < self.C - eps:
                    b = b2new
                else:
                    b = (b1new + b2new) / 2.0
                calcE()
                return 1

            a2 = a1
            if E[a1] >= 0:
                a2 = E.argmin()
            else:
                a2 = E.argmax()
            a2 = int(a2)
            if run(a2) == 0:
                a2 = randa2()
                return run(a2)
            else:
                return 1


        for round in range(max_round):
            tried, changed = 0, 0
            for i in range(N):
                if (y[i] * E[i] < -self.tol and alpha[i] < self.C - eps) or (
                    y[i] * E[i] > self.tol and alpha[i] > 0 + eps
                ):
                    tried += 1
                    changed += innerLoop(i)
            if tried == 0:
                break
            print(tried, changed, round)

        # generate model
        self.w = np.zeros(self.dim)
        for i in range(N):
            self.w += alpha[i] * y[i] * X[i]

        js = np.where(alpha > 0 + self.tol)[0]
        for j in js:
            self.b += y[j]
            for i in range(N):
                self.b -= alpha[i] * y[i] * K[i][j]
        self.b /= len(js)

    def predict(self, X):
        """
        Generate prediction probabilities on a new
        collection of data points by your model.
        """
        return np.sign(np.dot(X, self.w) + self.b)

In [610]:
class SVM2:
    def __init__(self, dim, ):
        """
        Adding other parameters is acceptable but not necessary.
        """

    def fit(self, X, y):
        """
        Fit the coefficients via your method2
        """
        
    def predict(self, X):
        """
        A same predict function with SVM1 is acceptable.
        """

# construct and train your models

In [611]:
# generate data
X_data, y_data, mislabel = generate_data(20, 10000) 

# split data
# import sklearn.model_selection

def train_test_split(X, y, test_rate):
    Xy = np.concatenate((X, y), axis=1)
    np.random.shuffle(Xy)
    tot_size = Xy.shape[0]
    test_size = int(test_rate * tot_size)
    return Xy[test_size:, 0:-1], Xy[0:test_size, 0:-1], Xy[test_size:, -1:], Xy[0:test_size, -1:]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_rate=0.2)

# X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_data, y_data, test_size=0.2)

X_train[:2], X_test[:2], y_train[:2], y_test[:2]

(array([[ -3.55120693,  -8.76758146,   2.46498197, -14.91545411,
           2.62565847,   7.62238294,  -6.26849473,   7.21598377,
           6.75655827,   1.1214684 ,  -4.83543785,  -3.48832063,
           0.97206736,   7.46857756,  -0.30649137, -14.31598174,
           5.4053784 ,   9.52740923, -13.01658403,  -2.01532135],
        [ -4.01574956, -17.56847336,  -0.04098731,  10.40260219,
          -0.70503513,  11.99551972,   2.95651098,  -9.07596244,
         -14.89557101,   1.09387988, -10.84820134,   7.29861279,
          -4.66034523, -10.80997853,  -8.97327815,  -0.18188037,
           5.57963657,  17.92117208,  -3.77329804,  -1.97646458]]),
 array([[ -2.02037772, -14.42332345,  -9.30418637, -11.03813262,
           6.56607582,  -7.67496152,  -3.10858564,  -1.99492008,
          -7.05828704,   5.60365988,  -8.4374468 ,   2.06341138,
          -4.88318659,  25.39816605,  15.85513766,   1.09302409,
           3.51733868,  -9.88689899,   2.72439621, -12.88484601],
        [  4.4854321

In [612]:
# construct model and train (remember to record your time consumption)
model1 = SVM1(dim=X_test.shape[1]) 
model1.fit(X_train, y_train)

2285 453 0
2362 499 1
2508 536 2
2749 664 3
2857 693 4


# predict and compare your results

In [613]:
# make prediction
tpred = model1.predict(X_train).reshape(y_train.shape)
pred = model1.predict(X_test).reshape(y_test.shape)

# compare with generated label

rate = np.count_nonzero(np.abs(pred - y_test) < 0.0005) / y_test.shape[0]
trate = np.count_nonzero(np.abs(tpred - y_train) < 0.0005) / y_train.shape[0]

print(rate, mislabel)
print(trate, mislabel)

# compare each method(in terms of accuracy, time consumption, etc.)

# (Optional) compare with sklearn


0.875 0.0391
0.882125 0.0391
